In [1]:
import pandas as pd
import json
import numpy as np
from pandas.io.json import json_normalize
from scipy.spatial.distance import cosine
import csv
import ast
import os

In [2]:

# loading the region-cell data
table = pd.read_csv('../../data/CDR/hash/intersect.csv', header = None) 
table.columns = ['region', 'proportions']
table.index = table.region
table.sort_values(['region'], inplace=True)

# loading the cell-proportion data
prop_table = pd.read_csv('../../data/CDR/hash/cell_intersect.csv', header = None) 
prop_table.columns = ['cell', 'proportions']
prop_table.index = prop_table.cell
prop_table.sort_values(['cell'], inplace=True)


In [3]:
prop_table.head()

,cell,proportions
cell,,
1,1,{}
2,2,{}
3,3,{}
4,4,{}
5,5,{}


In [ ]:


def get_cells_per_region(table, region_id):
    ids = table.iloc[region_id].proportions
    ids = ast.literal_eval(table.get_value(region_id, "proportions"))
    return ids.keys()

def get_call_data(source, dest, df):        
    source_dict = get_cells_per_region(table, source)
    dest_dict = get_cells_per_region(table, dest)

    subset = df[df.index.isin(source_dict)]
    subset.index = subset.dest
    subset = subset[subset.index.isin(dest_dict)]
            
    return subset

def calculate_actual_call(s_cell, d_cell, call, s_region, d_region):
    """
        Create another column on the subset DataFrame that is proportional to the regions.
    """
    source_prop = ast.literal_eval(prop_table.get_value(s_cell, "proportions"))
    dest_prop = ast.literal_eval(prop_table.get_value(d_cell, "proportions"))

    try:
        final = source_prop[str(s_region)] * dest_prop[str(d_region)] * call
    except:
        final = 0
    
    return final

In [ ]:
for f in os.listdir('cdr'):
    region_network = pd.DataFrame(columns=['time','source_region','dest_region','adjusted_call'])

    # loading the raw data
    df = pd.read_csv(os.getcwd() + '/cdr/' + f) 
    df.columns = ['index','time','source','dest','call']
    df.index = df.source
    
    for s in range(1,81):
        for d in range(1,81):
            # get a subset of records for the source and dest
            subdf = get_call_data(s, d, df)        
            subdf["source_region"] = s
            subdf["dest_region"] = d
            print (s, d)
            # create a column with adjusted call values
            try:
                subdf["adjusted_call"] = np.vectorize(calculate_actual_call)(subdf["source"], subdf["dest"], subdf["call"], subdf["source_region"], subdf["dest_region"])
            except:
                print ("error in making a column...")
                continue

            # do aggregation for 
            subdf = subdf.groupby("time").agg({
                        "source_region": "first",
                        "dest_region": "first",               
                        "adjusted_call": "sum"
                    })
            region_network = region_network.append(subdf)


    region_network.time = region_network.index        
    output_filename = '../../data/CDR/generated/region_network_' + f
    region_network.to_csv(output_filename, encoding='utf-8', index=False)

In [ ]:
### TEST Code
region_network = pd.DataFrame(columns=['time','source_region','dest_region','adjusted_call'])

# loading the raw data
df = pd.read_csv('../../data/CDR/MItoMI-2013-11-01.txt', delimiter='\t', header=None) 
df.columns = ['time','source','dest','call']
df.index = df.source

print ("file loaded.")

# change miliseconds to datetime
df.index = pd.to_datetime(df['time'],unit='ms',utc=True)
df.index = df.index.tz_localize('UTC').tz_convert('Europe/Rome')
df['hour'] = df.index.hour

df = df.groupby(['source', 'dest','hour'], as_index=False).agg({     
            'time': 'min',
            'call': 'sum'
        })

for s in range(1,81):
        for d in range(1,81):
            # get a subset of records for the source and dest
            subdf = get_call_data(s, d)        
            subdf["source_region"] = s
            subdf["dest_region"] = d
            print (s, d)
            # create a column with adjusted call values
            try:
                subdf["adjusted_call"] = np.vectorize(calculate_actual_call)(subdf["source"], subdf["dest"], subdf["call"], subdf["source_region"], subdf["dest_region"])
            except:
                print ("error in making a column...")
                continue

            # do aggregation for 
            subdf = subdf.groupby("hour").agg({
                        "source_region": "first",
                        "dest_region": "first",               
                        "adjusted_call": "sum"
                    })
            region_network = region_network.append(subdf)


    region_network.time = region_network.index        
    output_filename = '../../data/CDR/generated/region_network_' + f
    region_network.to_csv(output_filename, encoding='utf-8', index=False)

In [ ]:
region_network.head()